<a href="https://colab.research.google.com/github/minsoojo/MLprogramming/blob/main/1week/wisdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# 1. 데이터 불러오기
# df = pd.read_csv("/content/drive/MyDrive/3-2/MLprogramming/WISDM_ar_v1.1_raw.txt", header=None)
df = pd.read_csv(
    "/content/drive/MyDrive/3-2/MLprogramming/WISDM_ar_v1.1_raw.txt",
    header=None,
    names=["user", "activity", "timestamp", "x", "y", "z"],
    sep=",",          # 콤마로 먼저 나눔
    engine="python",
    on_bad_lines="skip"   # 잘못된 줄은 건너뛰기
)

# 각 행 끝에 붙은 ; 제거
df["z"] = df["z"].str.replace(";", "", regex=False).astype(float)

df.columns = ["user", "activity", "timestamp", "x", "y", "z"]

# 2. activity 라벨 인코딩
le = LabelEncoder()
df["label"] = le.fit_transform(df["activity"])

# 3. 슬라이딩 윈도우 (예: 200 샘플 = 10초) #오버래핑 > 정확성향상
window_size = 200
X, y = [], []
data = df[["x", "y", "z"]].values
labels = df["label"].values

for i in range(0, len(data) - window_size, window_size):
    X.append(data[i:i+window_size])
    y.append(labels[i+window_size-1])  # 마지막 라벨 사용

X = np.array(X)
y = to_categorical(np.array(y))

###############################################
# X, y = [], []
# window_size = 200

# for user_id, user_df in df.groupby("user"):
#     activities = user_df["label"].values
#     signals = user_df[["x", "y", "z"]].values

#     for i in range(0, len(signals) - window_size, window_size):
#         X.append(signals[i:i+window_size])
#         # 윈도우 안에서 가장 많이 나온 activity 라벨 사용
#         y.append(np.bincount(activities[i:i+window_size]).argmax())

# X = np.array(X)
# y = to_categorical(np.array(y))

# 4. train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. CNN 모델 정의
model = Sequential([
    Conv1D(64, 3, activation="relu", input_shape=(window_size, 3)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation="relu"),
    MaxPooling1D(2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(y.shape[1], activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 6. 학습
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# 7. 평가
loss, acc = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {acc:.4f}")

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.5289 - loss: 2.3029 - val_accuracy: 0.7001 - val_loss: nan
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7381 - loss: 0.6982 - val_accuracy: 0.7203 - val_loss: nan
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7886 - loss: 0.5519 - val_accuracy: 0.8215 - val_loss: nan
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8305 - loss: 0.4565 - val_accuracy: 0.8629 - val_loss: nan
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8548 - loss: 0.4042 - val_accuracy: 0.8114 - val_loss: nan
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8528 - loss: 0.3649 - val_accuracy: 0.8813 - val_loss: nan
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8873 - loss: 0.2862 - val_accuracy: 0.8859 - val_loss: nan
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9176 - loss: 0.2324 - val_accuracy: 0.8933 - val_loss: nan
Epoch 9/10
68/68 ━━━━━━━━━